In [ ]:
MODELBEGIN

//PART 3

EQUATION("Quality_Firm")
v[0] = CURRENT;
v[1] = V("q_min");
v[2] = V("q_max");
v[3] = uniform(v[1],v[2]);
v[4] = v[0]+v[3];
RESULT(v[4])

EQUATION("Competitiveness_Firm")
v[0] = V("Quality_Firm");
v[1] = V("Price_Firm");
v[2] = V("Quality_Elasticity");
v[3]= V("Price_Elasticity");
v[4]= pow(v[0],v[2])/pow(v[1],v[3]); //equação dinâmica
RESULT(v[4])


EQUATION("Market_Share") // market share está dentro da firma, mas vamos calcular o market share do setor também
v[0] = CURRENT;
v[1]=V("Competitiveness");
v[2]=AVES(PARENT, "Competitiveness"); // começa a fazer a busca a partir do obj acima, a macro PARENT especifica que quero o objeto acima
v[3]=V("ms_adjustment");
v[4]= v[0]*(1+v[3]*((v[1]/v[2])-1));
RESULT(v[4])

EQUATION("Consistency") // em termos de resultado, não precisamos dessa variável, mas ela é importante para checar a consistência. Vamos ter que implementar uma normalização para garantir que o MS seja um percentual entre zero e 1.
v[0] = SUM("Market_Share_Sector");
CYCLE(cur, "FIRM")
{
v[1]=VS(cur, "Market_Share");
WRITES(cur, "Market_Share", v[1]/v[0]); // reescrevendo o valor do Market Share
}
RESULT(SUM("Market_Share"))

EQUATION("Exit")
CYCLE_SAFE(cur, "FIRM")
{
v[0]=VS(cur, "Market_Share");
v[1]=V("ms_threshold");
if(v[0]<v[1])
DELETE(cur);
}
RESULT(0)

EQUATION("Entry")
v[0] = V("switch_entry");
if(v[0]==1)
{
v[1] = MAX("Market_Share");
cur = SEARCH_CND("Market_Share", v[1]);
ADDOBJ_EX("FIRM", cur);
}
if(v[0]==2)
{
cur=RNDDRAW_FAIR("FIRM");
ADDOBJ_EX("FIRM", cur);
}
RESULT(0)

EQUATION("Inverse_HHI")
v[0]=0;
CYCLE(cur, "FIRM")
{
v[1]=VS(cur, "Market_Share");
v[0]=v[0]+v[1]*v[1];
}
v[2]=v[0]!=0? 1/v[0] : 0; // versão compacta de if else
RESULT(v[2])



MODELEND